# CRACO-327 - metadata and CRACO-wcs dont agree.ipynb
The error was

2024-04-04 14:17:04,389 [skadi-17:1084570] r98 mpipipeline Exception running pipeline
Traceback (most recent call last):
  File "/CRACO/SOFTWARE/ban115/craco-python/src/craco/mpipipeline.py", line 1158, in _main
    processor.run()
  File "/CRACO/SOFTWARE/ban115/craco-python/src/craco/mpipipeline.py", line 1060, in run
    plan = PipelinePlan(adapter, self.obs_info.values, prev_plan=plan)
  File "/CRACO/SOFTWARE/ban115/craft/src/craft/craco_plan.py", line 561, in __init__
    craco_wcs = CracoWCS.from_plan(self)
  File "/CRACO/SOFTWARE/ban115/craft/src/craft/craco_wcs.py", line 131, in from_plan
    return CracoWCS(plan.phase_center,
  File "/CRACO/SOFTWARE/ban115/craft/src/craft/craco_wcs.py", line 84, in __init__
    assert altaz.alt.deg > 10, f'target is below ASKAP horizon {altaz} {lst}'
AssertionError: target is below ASKAP horizon <SkyCoord (AltAz: obstime=60404.29417975183, location=(-2556084.65961682, 5097398.3818179, -2848424.06141933) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (249.86985127, 9.97024392)> 3.698244555820127 hourangle


2nd error was
2024-04-04 14:59:35,598 [skadi-17:1115513] r98 mpipipeline Exception running pipeline
Traceback (most recent call last):
  File "/CRACO/SOFTWARE/ban115/craco-python/src/craco/mpipipeline.py", line 1158, in _main
    processor.run()
  File "/CRACO/SOFTWARE/ban115/craco-python/src/craco/mpipipeline.py", line 1060, in run
    plan = PipelinePlan(adapter, self.obs_info.values, prev_plan=plan)
  File "/CRACO/SOFTWARE/ban115/craft/src/craft/craco_plan.py", line 561, in __init__
    craco_wcs = CracoWCS.from_plan(self)
  File "/CRACO/SOFTWARE/ban115/craft/src/craft/craco_wcs.py", line 133, in from_plan
    return CracoWCS(plan.phase_center,
  File "/CRACO/SOFTWARE/ban115/craft/src/craft/craco_wcs.py", line 86, in __init__
    assert altaz.alt.deg > 10, f'target is below ASKAP horizon {altaz} {lst}'
AssertionError: target is below ASKAP horizon <SkyCoord (AltAz: obstime=60404.33350135183, location=(-2556084.65961682, 5097398.3818179, -2848424.06141933) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (223.12174179, 9.99225758)> 4.64454677131109 hourangle




In [63]:
from craco.metadatafile import MetadataFile
from pylab import *
from craft import craco_wcs
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy import units as u
from astropy.time import Time

In [64]:
mfile='/data/SKADI_00_0/craco/SB060752/scans/00/20240404061614/metafile.json'
mfile='/data/SKADI_00_0/craco/SB060755/scans/00/20240404065841/metafile.json'

In [65]:
m = MetadataFile(mfile)

In [78]:
log_obstime = 60404.29417975183
log_obstime = 60404.33350135183
obstime=Time(log_obstime, format='mjd', scale='tai')

In [79]:
# this is the card coded site in craco_wcs.py from - who knows where?
site = EarthLocation.from_geocentric(-2556084.65961682,
                                                     5097398.3818179,
                                                     -2848424.06141933,
                                                     u.meter)


In [80]:
obstime.location = site # needed to remove warning

In [81]:
lst = obstime.sidereal_time('apparent', site.lon)
lst

<Longitude 4.64454693 hourangle>

In [82]:
ncards=48
beamrank=98
#beam = beamrank - ncards
beam = 16
source = list(m.sources(beam).values())[0]
source

{'name': 'G348-50',
 'ra': 325.2747721270183,
 'dec': -46.17800668269095,
 'epoch': 'J2000',
 'skycoord': <SkyCoord (ICRS): (ra, dec) in deg
     (325.27477213, -46.17800668)>,
 'scan_times': [[<Time object: scale='tai' format='mjd' value=60404.290933631826>,
   <Time object: scale='tai' format='mjd' value=60404.29104883183>]]}

In [83]:
target = source['skycoord']
target

<SkyCoord (ICRS): (ra, dec) in deg
    (325.27477213, -46.17800668)>

In [84]:
altaz = target.transform_to(AltAz(obstime=obstime, location=site))
altaz

<SkyCoord (AltAz: obstime=60404.33350135183, location=(-2556084.65961682, 5097398.3818179, -2848424.06141933) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (223.1217417, 9.99225525)>

In [87]:
for beam in range(36):
    source = list(m.sources(beam).values())[0]
    target = source['skycoord']
    altaz = target.transform_to(AltAz(obstime=obstime, location=site))
    print(beam, altaz.alt.deg)


0 12.794703266503983
1 13.742297610969246
2 13.243959012532502
3 14.193342500093273
4 11.394616184639583
5 12.341648801720368
6 13.287504852522877
7 14.232430356317817
8 14.689099749499977
9 15.142071194905085
10 15.59134968779551
11 14.640645503808193
12 13.689423048150427
13 12.737428130355
14 12.293669418737176
15 11.846068126736746
16 9.992255246620847
17 10.939299654962346
18 11.88478274193546
19 12.828952364264724
20 13.772052126318444
21 14.714321907383002
22 15.176667965178984
23 15.635355262432094
24 16.090393760309592
25 16.541787500440158
26 16.989534570864123
27 16.036934301158894
28 15.08420670634343
29 14.131096495950374
30 13.177346450844475
31 12.222696830632657
32 11.784403692733777
33 11.342219192270234
34 10.896140043513093
35 10.446157076897588


In [88]:
(m.times[0] - obstime).to(u.minute) # how on earth did this happen?????

<Quantity -61.29751681 min>

In [89]:
m.times[0].iso

'2024-04-04 06:58:56.666'

In [90]:
obstime.iso

'2024-04-04 08:00:14.517'

In [91]:
mfile

'/data/SKADI_00_0/craco/SB060755/scans/00/20240404065841/metafile.json'